In [ ]:
pip install nfl_data_py


In [ ]:
import nfl_data_py as nfl
import pandas as pd

years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

games = nfl.import_schedules(years)

In [ ]:
# Data
import nfl_data_py as nfl
import pandas as pd

years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

games = nfl.import_schedules(years)
completed_games = games[games['result'] != '']

completed_games['winner'] = completed_games.apply(
    lambda row: row['home_team'] if row['home_score'] > row['away_score'] else row['away_team'],
    axis=1
)

home_wins = completed_games[completed_games['winner'] == completed_games['home_team']].groupby('home_team').size()
away_wins = completed_games[completed_games['winner'] == completed_games['away_team']].groupby('away_team').size()

win_counts = pd.DataFrame({
    'Home Wins': home_wins,
    'Away Wins': away_wins
}).fillna(0).astype(int)

win_counts = win_counts.reset_index().rename(columns={'index': 'Team'})

print(win_counts)

In [ ]:
#Bar graph
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt

years = list(range(2000, 2024))

games = nfl.import_schedules(years)

completed_games = games[games['result'] != '']

completed_games['winner'] = completed_games.apply(
    lambda row: row['home_team'] if row['home_score'] > row['away_score'] else row['away_team'],
    axis=1
)

home_wins = completed_games[completed_games['winner'] == completed_games['home_team']].groupby('home_team').size()
away_wins = completed_games[completed_games['winner'] == completed_games['away_team']].groupby('away_team').size()

win_counts = pd.DataFrame({
    'Home Wins': home_wins,
    'Away Wins': away_wins
}).fillna(0).astype(int)

win_counts = win_counts.reset_index().rename(columns={'index': 'Team'})

win_counts['Total Wins'] = win_counts['Home Wins'] + win_counts['Away Wins']
win_counts = win_counts.sort_values(by='Total Wins', ascending=False)

print(win_counts)

plt.figure(figsize=(14, 8))
win_counts.plot(kind='bar', x='Team', y=['Home Wins', 'Away Wins'], stacked=True, color=['blue', 'orange'])

plt.title('NFL Home vs. Away Wins (2000-2023)', fontsize=16)
plt.xlabel('Team', fontsize=12)
plt.ylabel('Number of Wins', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Win Type')
plt.tight_layout()
plt.show()

In [ ]:
#Heat map
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

years = list(range(2000, 2024))
games = nfl.import_schedules(years)

completed_games = games[games['result'] != '']

completed_games['winner'] = completed_games.apply(
    lambda row: row['home_team'] if row['home_score'] > row['away_score'] else row['away_team'],
    axis=1
)

completed_games['Season'] = completed_games['season']

home_wins = completed_games[completed_games['winner'] == completed_games['home_team']].groupby(['home_team', 'Season']).size().unstack(fill_value=0)
away_wins = completed_games[completed_games['winner'] == completed_games['away_team']].groupby(['away_team', 'Season']).size().unstack(fill_value=0)

home_wins.columns = [f"Home {col}" for col in home_wins.columns]
away_wins.columns = [f"Away {col}" for col in away_wins.columns]

win_counts = pd.concat([home_wins, away_wins], axis=1).fillna(0).astype(int)

plt.figure(figsize=(14, 10))
sns.heatmap(win_counts, annot=True, fmt="d", cmap="coolwarm", linewidths=0.5, cbar=True)

plt.title('NFL Home and Away Wins by Team and Season (2000-2023)', fontsize=16)
plt.xlabel('Season and Win Type', fontsize=12)
plt.ylabel('Team', fontsize=12)
plt.tight_layout()

plt.show()

In [ ]:
#Dot plot
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

years = list(range(2000, 2024))

games = nfl.import_schedules(years)

completed_games = games[games['result'] != '']

home_wins = completed_games[completed_games['home_score'] > completed_games['away_score']].groupby('home_team').size()
away_wins = completed_games[completed_games['away_score'] > completed_games['home_score']].groupby('away_team').size()

win_counts = pd.DataFrame({
    'Home Wins': home_wins,
    'Away Wins': away_wins
}).fillna(0).astype(int)

win_counts['Total Wins'] = win_counts['Home Wins'] + win_counts['Away Wins']
win_counts = win_counts.sort_values(by='Total Wins', ascending=False)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 10), sharex=True)

axes[0].scatter(win_counts.index, win_counts['Home Wins'], color='blue', s=100)
axes[0].set_title('Home Wins (2000-2023)', fontsize=14)
axes[0].set_ylabel('Number of Wins', fontsize=12)
axes[0].tick_params(axis='x', rotation=90)

x_home = np.arange(len(win_counts.index)).reshape(-1, 1)
y_home = win_counts['Home Wins'].values.reshape(-1, 1)
home_model = LinearRegression()
home_model.fit(x_home, y_home)
y_home_pred = home_model.predict(x_home)
axes[0].plot(win_counts.index, y_home_pred, color='red', linewidth=2, label='Trend Line')
axes[0].legend()

axes[1].scatter(win_counts.index, win_counts['Away Wins'], color='orange', s=100)
axes[1].set_title('Away Wins (2000-2023)', fontsize=14)
axes[1].set_xlabel('Team', fontsize=12)
axes[1].set_ylabel('Number of Wins', fontsize=12)
axes[1].tick_params(axis='x', rotation=90)

x_away = np.arange(len(win_counts.index)).reshape(-1, 1)
y_away = win_counts['Away Wins'].values.reshape(-1, 1)
away_model = LinearRegression()
away_model.fit(x_away, y_away)
y_away_pred = away_model.predict(x_away)
axes[1].plot(win_counts.index, y_away_pred, color='red', linewidth=2, label='Trend Line')
axes[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
#Bubble chart
import nfl_data_py as nfl
import pandas as pd
import matplotlib.pyplot as plt

years = list(range(2000, 2024))
games = nfl.import_schedules(years)

completed_games = games[games['result'] != '']
home_wins = completed_games[completed_games['home_score'] > completed_games['away_score']].groupby('home_team').size()
away_wins = completed_games[completed_games['away_score'] > completed_games['home_score']].groupby('away_team').size()
win_counts = pd.DataFrame({
    'Home Wins': home_wins,
    'Away Wins': away_wins
}).fillna(0).astype(int)

win_counts['Total Wins'] = win_counts['Home Wins'] + win_counts['Away Wins']

plt.figure(figsize=(12, 8))
bubble_sizes = win_counts['Total Wins'] * 10
plt.scatter(
    win_counts['Home Wins'],
    win_counts['Away Wins'],
    s=bubble_sizes,
    alpha=0.7,
    edgecolors="black",
    c=win_counts['Total Wins'],
    cmap='coolwarm'
)

for team, row in win_counts.iterrows():
    plt.text(
        row['Home Wins'],
        row['Away Wins'],
        team,
        fontsize=9,
        ha='center',
        va='center'
    )

plt.title('NFL Home vs Away Wins (2000-2023)', fontsize=16)
plt.xlabel('Home Wins', fontsize=12)
plt.ylabel('Away Wins', fontsize=12)
plt.colorbar(label='Total Wins')
plt.grid(alpha=0.3)
plt.tight_layout()

plt.show()

In [ ]:
#linear regression
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import nfl_data_py as nfl

years = list(range(2000, 2024))

games = nfl.import_schedules(years)

completed_games = games[games['result'] != '']

completed_games['winner'] = completed_games.apply(
    lambda row: row['home_team'] if row['home_score'] > row['away_score'] else row['away_team'],
    axis=1
)

home_wins = completed_games[completed_games['winner'] == completed_games['home_team']].groupby('home_team').size()
away_wins = completed_games[completed_games['winner'] == completed_games['away_team']].groupby('away_team').size()

win_counts = pd.DataFrame({
    'Home Wins': home_wins,
    'Away Wins': away_wins
}).fillna(0).astype(int)

X = win_counts[['Home Wins']].values
y = win_counts['Away Wins'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

plt.figure(figsize=(10, 6))
plt.scatter(X, y, color='blue', label='Actual Data')
plt.plot(X, model.predict(X), color='red', linewidth=2, label='Regression Line')
plt.title('Linear Regression: Home Wins vs Away Wins (2000-2023)')
plt.xlabel('Home Wins')
plt.ylabel('Away Wins')
plt.legend()
plt.show()